# Function Calling with Mistral Model


In [1]:
!pip install pandas "mistralai>=0.1.2"

/usr/bin/sh: 1: pip: not found


### Load API key

In [2]:
from helper import load_mistral_api_key
api_key, dlai_endpoint = load_mistral_api_key(ret_key=True)

In [3]:
import pandas as pd

In [4]:
data = {
    "transaction_id": ["T1001", "T1002", "T1003", "T1004", "T1005"],
    "customer_id": ["C001", "C002", "C003", "C002", "C001"],
    "payment_amount": [125.50, 89.99, 120.00, 54.30, 210.20],
    "payment_date": [
        "2021-10-05",
        "2021-10-06",
        "2021-10-07",
        "2021-10-05",
        "2021-10-08",
    ],
    "payment_status": ["Paid", "Unpaid", "Paid", "Paid", "Pending"],
}
df = pd.DataFrame(data)

In [5]:
df

,transaction_id,customer_id,payment_amount,payment_date,payment_status
0,T1001,C001,125.50,2021-10-05,Paid
1,T1002,C002,89.99,2021-10-06,Unpaid
2,T1003,C003,120.00,2021-10-07,Paid
3,T1004,C002,54.30,2021-10-05,Paid
4,T1005,C001,210.20,2021-10-08,Pending


#### How you might answer data questions without function calling
- Not recommended, but an example to serve as a contrast to function calling.

In [6]:
data = """
    "transaction_id": ["T1001", "T1002", "T1003", "T1004", "T1005"],
    "customer_id": ["C001", "C002", "C003", "C002", "C001"],
    "payment_amount": [125.50, 89.99, 120.00, 54.30, 210.20],
    "payment_date": [
        "2021-10-05",
        "2021-10-06",
        "2021-10-07",
        "2021-10-05",
        "2021-10-08",
    ],
    "payment_status": ["Paid", "Unpaid", "Paid", "Paid", "Pending"],
}
"""
transaction_id = "T1001"

prompt = f"""
Given the following data, what is the payment status for \
 transaction_id={transaction_id}?

data:
{data}

"""

In [7]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage


def mistral(user_message, model="mistral-small-latest", is_json=False):
    client = MistralClient(api_key=api_key, endpoint=dlai_endpoint)
    messages = [ChatMessage(role="user", content=user_message)]

    if is_json:
        chat_response = client.chat(
            model=model, messages=messages, response_format={"type": "json_object"}
        )
    else:
        chat_response = client.chat(model=model, messages=messages)

    return chat_response.choices[0].message.content

In [8]:
response = mistral(prompt)
print(response)

KeyboardInterrupt: 

## Step 1. User: specify tools and query

### Tools

- You can define all tools that you might want the model to call.

In [9]:
import json

In [10]:
def retrieve_payment_status(df: data, transaction_id: str) -> str:
    if transaction_id in df.transaction_id.values:
        return json.dumps(
            {"status": df[df.transaction_id == transaction_id].payment_status.item()}
        )
    return json.dumps({"error": "transaction id not found."})

In [11]:
status = retrieve_payment_status(df, transaction_id="T1001")
print(status)

{"status": "Paid"}


In [12]:
type(status)

str

In [13]:
def retrieve_payment_date(df: data, transaction_id: str) -> str:
    if transaction_id in df.transaction_id.values:
        return json.dumps(
            {"date": df[df.transaction_id == transaction_id].payment_date.item()}
        )
    return json.dumps({"error": "transaction id not found."})

In [14]:
date = retrieve_payment_date(df, transaction_id="T1002")
print(date)

{"date": "2021-10-06"}


- You can outline the function specifications with a JSON schema.

In [15]:
tool_payment_status = {
    "type": "function",
    "function": {
        "name": "retrieve_payment_status",
        "description": "Get payment status of a transaction",
        "parameters": {
            "type": "object",
            "properties": {
                "transaction_id": {
                    "type": "string",
                    "description": "The transaction id.",
                }
            },
            "required": ["transaction_id"],
        },
    },
}

In [16]:
type(tool_payment_status)

dict

In [17]:
tool_payment_date = {
    "type": "function",
    "function": {
        "name": "retrieve_payment_date",
        "description": "Get payment date of a transaction",
        "parameters": {
            "type": "object",
            "properties": {
                "transaction_id": {
                    "type": "string",
                    "description": "The transaction id.",
                }
            },
            "required": ["transaction_id"],
        },
    },
}

In [18]:
type(tool_payment_status)

dict

In [19]:
tools = [tool_payment_status, tool_payment_date]

In [20]:
type(tools)

list

In [21]:
tools

[{'type': 'function',
  'function': {'name': 'retrieve_payment_status',
   'description': 'Get payment status of a transaction',
   'parameters': {'type': 'object',
    'properties': {'transaction_id': {'type': 'string',
      'description': 'The transaction id.'}},
    'required': ['transaction_id']}}},
 {'type': 'function',
  'function': {'name': 'retrieve_payment_date',
   'description': 'Get payment date of a transaction',
   'parameters': {'type': 'object',
    'properties': {'transaction_id': {'type': 'string',
      'description': 'The transaction id.'}},
    'required': ['transaction_id']}}}]

### functools

In [22]:
import functools

In [23]:
names_to_functions = {
    "retrieve_payment_status": functools.partial(retrieve_payment_status, df=df),
    "retrieve_payment_date": functools.partial(retrieve_payment_date, df=df),
}

In [24]:
names_to_functions["retrieve_payment_status"](transaction_id="T1001")

'{"status": "Paid"}'

In [25]:
tools

[{'type': 'function',
  'function': {'name': 'retrieve_payment_status',
   'description': 'Get payment status of a transaction',
   'parameters': {'type': 'object',
    'properties': {'transaction_id': {'type': 'string',
      'description': 'The transaction id.'}},
    'required': ['transaction_id']}}},
 {'type': 'function',
  'function': {'name': 'retrieve_payment_date',
   'description': 'Get payment date of a transaction',
   'parameters': {'type': 'object',
    'properties': {'transaction_id': {'type': 'string',
      'description': 'The transaction id.'}},
    'required': ['transaction_id']}}}]

### User query

- Example: “What’s the status of my transaction?”

In [26]:
from mistralai.models.chat_completion import ChatMessage

chat_history = [
    ChatMessage(role="user", content="What's the status of my transaction?")
]

## Step 2. Model: Generate function arguments 

In [27]:
from mistralai.client import MistralClient

model = "mistral-large-latest"

client = MistralClient(api_key=os.getenv("MISTRAL_API_KEY"), endpoint=os.getenv("DLAI_MISTRAL_API_ENDPOINT"))

response = client.chat(
    model=model, messages=chat_history, tools=tools, tool_choice="auto"
)

response

ChatCompletionResponse(id='68e1d8260a6249e1bd4fd2aa16232870', object='chat.completion', created=1747918739, model='mistral-large-latest', choices=[ChatCompletionResponseChoice(index=0, message=ChatMessage(role='assistant', content='Sure, I can help with that. Please provide the transaction ID.', name=None, tool_calls=None), finish_reason=<FinishReason.stop: 'stop'>)], usage=UsageInfo(prompt_tokens=161, total_tokens=176, completion_tokens=15))

In [28]:
response.choices[0].message.content

'Sure, I can help with that. Please provide the transaction ID.'

### Save the chat history

In [29]:
chat_history.append(
    ChatMessage(role="assistant", content=response.choices[0].message.content)
)
chat_history.append(ChatMessage(role="user", content="My transaction ID is T1001."))
chat_history

[ChatMessage(role='user', content="What's the status of my transaction?", name=None, tool_calls=None),
 ChatMessage(role='assistant', content='Sure, I can help with that. Please provide the transaction ID.', name=None, tool_calls=None),
 ChatMessage(role='user', content='My transaction ID is T1001.', name=None, tool_calls=None)]

In [30]:
response = client.chat(
    model=model, messages=chat_history, tools=tools, tool_choice="auto"
)

In [31]:
response

ChatCompletionResponse(id='8e3e1f8ef011420db41e2571d16fd0ff', object='chat.completion', created=1747918827, model='mistral-large-latest', choices=[ChatCompletionResponseChoice(index=0, message=ChatMessage(role='assistant', content='', name=None, tool_calls=[ToolCall(id='eMTZiTinV', type=<ToolType.function: 'function'>, function=FunctionCall(name='retrieve_payment_status', arguments='{"transaction_id": "T1001"}'))]), finish_reason=<FinishReason.tool_calls: 'tool_calls'>)], usage=UsageInfo(prompt_tokens=188, total_tokens=219, completion_tokens=31))

In [32]:
response.choices[0].message

ChatMessage(role='assistant', content='', name=None, tool_calls=[ToolCall(id='eMTZiTinV', type=<ToolType.function: 'function'>, function=FunctionCall(name='retrieve_payment_status', arguments='{"transaction_id": "T1001"}'))])

In [33]:
chat_history.append(response.choices[0].message)

- Notice these fields:
- `name='retrieve_payment_status'`
- `arguments='{"transaction_id": "T1001"}'`

## Step 3. User: Execute function to obtain tool results

- Currently, the user is the one who will execute these functions (the model will not execute these functions on its own).

In [34]:
tool_function = response.choices[0].message.tool_calls[0].function
print(tool_function)

name='retrieve_payment_status' arguments='{"transaction_id": "T1001"}'


In [35]:
tool_function.name

'retrieve_payment_status'

In [36]:
tool_function.arguments

'{"transaction_id": "T1001"}'

- The function arguments are expected to be in a Python dictionary and not a string.
- To make this string into a dictionary, you can use `json.loads()`  

In [37]:
args = json.loads(tool_function.arguments)
print(args)

{'transaction_id': 'T1001'}


- Recall the functools dictionary that you made earlier

```Python
import functools
names_to_functions = {
    "retrieve_payment_status": 
      functools.partial(retrieve_payment_status, df=df),
    
    "retrieve_payment_date": 
      functools.partial(retrieve_payment_date, df=df),
}
```

In [38]:
function_result = names_to_functions[tool_function.name](**args)
function_result

'{"status": "Paid"}'

- The output of the function call can be saved as a chat message, with the role "tool".

> Note: This notebook was updated in January 2025. The primary change involves defining the 'tool,' where the code now uses tool_call_id instead of tool_msg. This update aligns with the latest API requirements, as earlier versions did not require tool_call_id.

The original code (as shown in the video) can be found below:

```python
tool_msg = ChatMessage(role="tool", name=tool_function.name, content=function_result)
chat_history.append(tool_msg)
```

In [39]:
tool_call_id = response.choices[0].message.tool_calls[0].id
chat_history.append(
    {"role":"tool", 
     "name":tool_function.name, 
     "content":function_result, 
     "tool_call_id":tool_call_id
    })

In [40]:
chat_history

[ChatMessage(role='user', content="What's the status of my transaction?", name=None, tool_calls=None),
 ChatMessage(role='assistant', content='Sure, I can help with that. Please provide the transaction ID.', name=None, tool_calls=None),
 ChatMessage(role='user', content='My transaction ID is T1001.', name=None, tool_calls=None),
 ChatMessage(role='assistant', content='', name=None, tool_calls=[ToolCall(id='eMTZiTinV', type=<ToolType.function: 'function'>, function=FunctionCall(name='retrieve_payment_status', arguments='{"transaction_id": "T1001"}'))]),
 {'role': 'tool',
  'name': 'retrieve_payment_status',
  'content': '{"status": "Paid"}',
  'tool_call_id': 'eMTZiTinV'}]

## Step 4. Model: Generate final answer
- The model can now reply to the user query, given the information provided by the "tool".

In [41]:
response = client.chat(model=model, messages=chat_history)
response.choices[0].message.content

'Great news! Your transaction with ID T1001 is marked as "Paid." If you have any further questions or need additional assistance, feel free to ask.'

**Asking for both status and date**

In [43]:
response = client.chat(
        model=model,
        messages=[
            ChatMessage(
                role="user",
                content="what is the status and data of my transaction ID T1001"
            )
        ],
    tools=tools,
    tool_choice="auto"

)

response

ChatCompletionResponse(id='5d55695ed2c143af8b58f34e24730a3a', object='chat.completion', created=1747919186, model='mistral-large-latest', choices=[ChatCompletionResponseChoice(index=0, message=ChatMessage(role='assistant', content='', name=None, tool_calls=[ToolCall(id='B89ytQ4tN', type=<ToolType.function: 'function'>, function=FunctionCall(name='retrieve_payment_status', arguments='{"transaction_id": "T1001"}')), ToolCall(id='9kx7jfz4R', type=<ToolType.function: 'function'>, function=FunctionCall(name='retrieve_payment_date', arguments='{"transaction_id": "T1001"}'))]), finish_reason=<FinishReason.tool_calls: 'tool_calls'>)], usage=UsageInfo(prompt_tokens=167, total_tokens=225, completion_tokens=58))